In [4]:
import os
import glob
import shutil
from sklearn.model_selection import train_test_split

# Base directory paths
BASE_DIR = r"C:\Users\ISU\cv_dataset\os_dataset"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR = os.path.join(BASE_DIR, "val")
TEST_DIR = os.path.join(BASE_DIR, "test")

# Ensure all directories exist
os.makedirs(os.path.join(TRAIN_DIR, "images"), exist_ok=True)
os.makedirs(os.path.join(TRAIN_DIR, "masks"), exist_ok=True)
os.makedirs(os.path.join(VAL_DIR, "images"), exist_ok=True)
os.makedirs(os.path.join(VAL_DIR, "masks"), exist_ok=True)
os.makedirs(os.path.join(TEST_DIR, "images"), exist_ok=True)

# Define patterns to locate files (focus on *_cropped.nii and *_seg.nii files)
image_pattern = "*_cropped.nii"  # Only the image files, without the *_seg_endpoints
mask_pattern = "*_seg.nii"  # Only the mask files (exclude *_seg_endpoints)

# Unzipped files
image_files = sorted(glob.glob(os.path.join(BASE_DIR, image_pattern)))
mask_files = sorted(glob.glob(os.path.join(BASE_DIR, mask_pattern)))

# Debugging: Print found files
print("Image Files:", image_files)
print("Mask Files:", mask_files)
print("Number of images:", len(image_files))
print("Number of masks:", len(mask_files))

# Ensure the number of images matches masks
assert len(image_files) == len(mask_files), "Mismatch between image and mask files."

# Split dataset into train, val, and test
train_img, temp_img, train_mask, temp_mask = train_test_split(image_files, mask_files, test_size=0.3, random_state=42)
val_img, test_img, val_mask, test_mask = train_test_split(temp_img, temp_mask, test_size=0.5, random_state=42)

# Helper function to move files
def move_files(image_list, mask_list, dest_dir, for_test=False):
    if not for_test:
        for img, mask in zip(image_list, mask_list):
            shutil.move(img, os.path.join(dest_dir, "images", os.path.basename(img)))
            shutil.move(mask, os.path.join(dest_dir, "masks", os.path.basename(mask)))
    else:
        for img in image_list:
            shutil.move(img, os.path.join(dest_dir, "images", os.path.basename(img)))

# Organize files into train, val, and test folders
move_files(train_img, train_mask, TRAIN_DIR)
move_files(val_img, val_mask, VAL_DIR)
move_files(test_img, [], TEST_DIR, for_test=True)

print("Dataset successfully organized!")


Image Files: []
Mask Files: ['C:\\Users\\ISU\\cv_dataset\\os_dataset\\pat0_cropped_seg.nii', 'C:\\Users\\ISU\\cv_dataset\\os_dataset\\pat12_cropped_seg.nii', 'C:\\Users\\ISU\\cv_dataset\\os_dataset\\pat14_cropped_seg.nii', 'C:\\Users\\ISU\\cv_dataset\\os_dataset\\pat17_cropped_seg.nii', 'C:\\Users\\ISU\\cv_dataset\\os_dataset\\pat20_cropped_seg.nii', 'C:\\Users\\ISU\\cv_dataset\\os_dataset\\pat25_cropped_seg.nii', 'C:\\Users\\ISU\\cv_dataset\\os_dataset\\pat50_cropped_seg.nii', 'C:\\Users\\ISU\\cv_dataset\\os_dataset\\pat55_cropped_seg.nii', 'C:\\Users\\ISU\\cv_dataset\\os_dataset\\pat59_cropped_seg.nii']
Number of images: 0
Number of masks: 9
Dataset successfully organized!


In [12]:
import os
import glob
import nibabel as nib

# Base directories (already set in your previous code)
BASE_DIR = r"C:\Users\ISU\cv_dataset\os_dataset"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR = os.path.join(BASE_DIR, "val")
TEST_DIR = os.path.join(BASE_DIR, "test")

# Print the base and dataset directory paths
print(f"Base Directory: {BASE_DIR}")
print(f"Training Directory: {TRAIN_DIR}")
print(f"Validation Directory: {VAL_DIR}")
print(f"Test Directory: {TEST_DIR}")

# Get all image and mask paths for training, validation, and testing
train_img_paths = glob.glob(os.path.join(TRAIN_DIR, "images", "*.nii"))  # Change to *.nii
train_mask_paths = glob.glob(os.path.join(TRAIN_DIR, "masks", "*.nii"))  # Change to *.nii
val_img_paths = glob.glob(os.path.join(VAL_DIR, "images", "*.nii"))  # Change to *.nii
val_mask_paths = glob.glob(os.path.join(VAL_DIR, "masks", "*.nii"))  # Change to *.nii
test_img_paths = glob.glob(os.path.join(TEST_DIR, "images", "*.nii"))  # Change to *.nii

# Debug: Print out the found image paths to verify if glob is working correctly
print("Training Images Paths:", train_img_paths)
print("Training Masks Paths:", train_mask_paths)
print("Validation Images Paths:", val_img_paths)
print("Validation Masks Paths:", val_mask_paths)
print("Testing Images Paths:", test_img_paths)

# Check the number of files in each dataset and display the results
print(f"Number of training images found: {len(train_img_paths)}")
print(f"Number of training masks found: {len(train_mask_paths)}")
print(f"Number of validation images found: {len(val_img_paths)}")
print(f"Number of validation masks found: {len(val_mask_paths)}")
print(f"Number of testing images found: {len(test_img_paths)}")

# Check directory contents for the images and masks to confirm the files are in place
print(f"Contents of {TRAIN_DIR}/images: ", os.listdir(os.path.join(TRAIN_DIR, "images")))
print(f"Contents of {TRAIN_DIR}/masks: ", os.listdir(os.path.join(TRAIN_DIR, "masks")))
print(f"Contents of {VAL_DIR}/images: ", os.listdir(os.path.join(VAL_DIR, "images")))
print(f"Contents of {VAL_DIR}/masks: ", os.listdir(os.path.join(VAL_DIR, "masks")))
print(f"Contents of {TEST_DIR}/images: ", os.listdir(os.path.join(TEST_DIR, "images")))

# Verify the accessibility of a sample image by loading it with nibabel
if train_img_paths:
    img_test = nib.load(train_img_paths[0])
    print(f"Test image shape: {img_test.shape}")
else:
    print("No training images found for test loading.")

# Create Dataset instances (this will only work if images and masks are found)
if len(train_img_paths) > 0 and len(train_mask_paths) > 0:
    train_dataset = HeartDataset(train_img_paths, train_mask_paths)
    print(f"Training dataset size: {len(train_dataset)}")
else:
    print("Training dataset is empty.")

if len(val_img_paths) > 0 and len(val_mask_paths) > 0:
    val_dataset = HeartDataset(val_img_paths, val_mask_paths)
    print(f"Validation dataset size: {len(val_dataset)}")
else:
    print("Validation dataset is empty.")

if len(test_img_paths) > 0:
    test_dataset = HeartDataset(test_img_paths)
    print(f"Test dataset size: {len(test_dataset)}")
else:
    print("Test dataset is empty.")


Base Directory: C:\Users\ISU\cv_dataset\os_dataset
Training Directory: C:\Users\ISU\cv_dataset\os_dataset\train
Validation Directory: C:\Users\ISU\cv_dataset\os_dataset\val
Test Directory: C:\Users\ISU\cv_dataset\os_dataset\test
Training Images Paths: ['C:\\Users\\ISU\\cv_dataset\\os_dataset\\train\\images\\pat10_cropped.nii', 'C:\\Users\\ISU\\cv_dataset\\os_dataset\\train\\images\\pat11_cropped.nii', 'C:\\Users\\ISU\\cv_dataset\\os_dataset\\train\\images\\pat13_cropped.nii', 'C:\\Users\\ISU\\cv_dataset\\os_dataset\\train\\images\\pat16_cropped.nii', 'C:\\Users\\ISU\\cv_dataset\\os_dataset\\train\\images\\pat18_cropped.nii', 'C:\\Users\\ISU\\cv_dataset\\os_dataset\\train\\images\\pat19_cropped.nii', 'C:\\Users\\ISU\\cv_dataset\\os_dataset\\train\\images\\pat1_cropped.nii', 'C:\\Users\\ISU\\cv_dataset\\os_dataset\\train\\images\\pat22_cropped.nii', 'C:\\Users\\ISU\\cv_dataset\\os_dataset\\train\\images\\pat23_cropped.nii', 'C:\\Users\\ISU\\cv_dataset\\os_dataset\\train\\images\\pat24_cr

In [1]:
import os
import glob
import torch
from torch.utils.data import Dataset, DataLoader
import nibabel as nib
import scipy.ndimage
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define configuration parameters
INPUT_SHAPE = (128, 128, 64)  # The target shape after resizing
BATCH_SIZE = 2
EPOCHS = 5
LEARNING_RATE = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Base directories (already set in your previous code)
BASE_DIR = r"C:\Users\ISU\cv_dataset\os_dataset"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR = os.path.join(BASE_DIR, "val")
# No need for the test directory since we are using a single patient image for testing
PATIENT_IMAGE_PATH = r"C:\Users\ISU\cv_dataset\patient_data\preprocessed_patient_image.nii.gz"
OUTPUT_MASK_PATH = r"C:\Users\ISU\cv_dataset\patient_data\segmented_mask.nii.gz"

# Create Dataset class to load images and masks
class HeartDataset(Dataset):
    def __init__(self, image_paths, mask_paths=None, transform=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def resize_3d(self, img, shape):
        zoom_factors = [shape[i] / img.shape[i] for i in range(3)]
        return scipy.ndimage.zoom(img, zoom_factors, order=1)

    def __getitem__(self, idx):
        img = nib.load(self.image_paths[idx]).get_fdata()
        img = (img - img.min()) / (img.max() - img.min())  # Normalize the image
        img = self.resize_3d(img, INPUT_SHAPE)
        img = torch.tensor(img, dtype=torch.float32).unsqueeze(0)  # Add channel dimension

        if self.mask_paths:
            mask = nib.load(self.mask_paths[idx]).get_fdata()
            mask = self.resize_3d(mask, INPUT_SHAPE)
            
            # Convert mask to binary (0 or 1)
            mask = (mask > 0).astype(np.float32)
            mask = torch.tensor(mask, dtype=torch.float32).unsqueeze(0)  # Add channel dimension
            
            return img, mask
        return img

# Get all image and mask paths for training and validation
train_img_paths = glob.glob(os.path.join(TRAIN_DIR, "images", "*.nii"))  # Change to *.nii
train_mask_paths = glob.glob(os.path.join(TRAIN_DIR, "masks", "*.nii"))  # Change to *.nii
val_img_paths = glob.glob(os.path.join(VAL_DIR, "images", "*.nii"))  # Change to *.nii
val_mask_paths = glob.glob(os.path.join(VAL_DIR, "masks", "*.nii"))  # Change to *.nii

# Create datasets for training and validation
train_dataset = HeartDataset(train_img_paths, train_mask_paths)
val_dataset = HeartDataset(val_img_paths, val_mask_paths)

# Create DataLoader objects for batching
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Define and train the model (UNet)
class UNet3D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet3D, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv3d(in_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=2)
        )
        self.decoder = nn.Sequential(
            nn.Conv3d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(64, out_channels, kernel_size=3, padding=1),
            nn.Sigmoid()
        )
        self.upconv = nn.ConvTranspose3d(64, 64, kernel_size=2, stride=2)

    def forward(self, x):
        enc = self.encoder(x)
        dec = self.upconv(enc)
        dec = self.decoder(dec)
        return dec

# Initialize model, loss function, and optimizer
model = UNet3D(in_channels=1, out_channels=1).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_function = nn.BCELoss()

# Training loop
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for images, masks in train_loader:
        images, masks = images.to(DEVICE), masks.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, masks)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {epoch_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), os.path.join(BASE_DIR, "unet_heart_segmentation_3d.pth"))

# --- Inference on Patient Image ---
def predict_patient_image(model, patient_image_path, output_mask_path, device):
    model.eval()
    # Load and preprocess patient image
    img = nib.load(patient_image_path).get_fdata()
    img = (img - img.min()) / (img.max() - img.min())  # Normalize the image
    img = scipy.ndimage.zoom(img, [float(dim) / img.shape[i] for i, dim in enumerate(INPUT_SHAPE)])
    img = torch.tensor(img, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)  # Add channel dimension

    with torch.no_grad():
        # Predict segmentation mask
        pred = model(img)
        pred = (pred > 0.5).float().cpu().numpy().squeeze()  # Apply threshold

    # Save the predicted mask as a NIfTI image
    pred_img = nib.Nifti1Image(pred, np.eye(4))
    nib.save(pred_img, output_mask_path)
    print(f"Prediction saved to: {output_mask_path}")

# Predict on the single patient image
predict_patient_image(model, PATIENT_IMAGE_PATH, OUTPUT_MASK_PATH, DEVICE)


Epoch 1/5, Loss: 13.9252
Epoch 2/5, Loss: 10.7028
Epoch 3/5, Loss: 7.7665
Epoch 4/5, Loss: 5.3920
Epoch 5/5, Loss: 4.6838
Prediction saved to: C:\Users\ISU\cv_dataset\patient_data\segmented_mask.nii.gz


# Latest code trying with slices

In [2]:
import os
import glob
import torch
from torch.utils.data import Dataset, DataLoader
import nibabel as nib
import scipy.ndimage
import torch.nn as nn
import torch.optim as optim
import numpy as np
import nrrd  # Library for handling .nrrd files

# Define configuration parameters
INPUT_SHAPE = (128, 128, 64)  # The target shape after resizing
BATCH_SIZE = 2
EPOCHS = 20
LEARNING_RATE = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Base directories
BASE_DIR = r"C:\Users\ISU\cv_dataset\os_dataset"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR = os.path.join(BASE_DIR, "val")

# Updated path for testing with your .nrrd file
PATIENT_IMAGE_PATH = r"C:\Users\ISU\cv_dataset\slice_data\4 Unnamed Series_1.nrrd"
OUTPUT_MASK_PATH = r"C:\Users\ISU\cv_dataset\slice_data\segmented_mask.nii.gz"

# Create Dataset class to load images and masks
class HeartDataset(Dataset):
    def __init__(self, image_paths, mask_paths=None, transform=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def resize_3d(self, img, shape):
        zoom_factors = [shape[i] / img.shape[i] for i in range(3)]
        return scipy.ndimage.zoom(img, zoom_factors, order=1)

    def __getitem__(self, idx):
        img = nib.load(self.image_paths[idx]).get_fdata()
        img = (img - img.min()) / (img.max() - img.min())  # Normalize the image
        img = self.resize_3d(img, INPUT_SHAPE)
        img = torch.tensor(img, dtype=torch.float32).unsqueeze(0)  # Add channel dimension

        if self.mask_paths:
            mask = nib.load(self.mask_paths[idx]).get_fdata()
            mask = self.resize_3d(mask, INPUT_SHAPE)
            
            # Convert mask to binary (0 or 1)
            mask = (mask > 0).astype(np.float32)
            mask = torch.tensor(mask, dtype=torch.float32).unsqueeze(0)  # Add channel dimension
            
            return img, mask
        return img

# Get all image and mask paths for training and validation
train_img_paths = glob.glob(os.path.join(TRAIN_DIR, "images", "*.nii"))
train_mask_paths = glob.glob(os.path.join(TRAIN_DIR, "masks", "*.nii"))
val_img_paths = glob.glob(os.path.join(VAL_DIR, "images", "*.nii"))
val_mask_paths = glob.glob(os.path.join(VAL_DIR, "masks", "*.nii"))

# Create datasets for training and validation
train_dataset = HeartDataset(train_img_paths, train_mask_paths)
val_dataset = HeartDataset(val_img_paths, val_mask_paths)

# Create DataLoader objects for batching
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Define and train the model (UNet)
class UNet3D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet3D, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv3d(in_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=2)
        )
        self.decoder = nn.Sequential(
            nn.Conv3d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(64, out_channels, kernel_size=3, padding=1),
            nn.Sigmoid()
        )
        self.upconv = nn.ConvTranspose3d(64, 64, kernel_size=2, stride=2)

    def forward(self, x):
        enc = self.encoder(x)
        dec = self.upconv(enc)
        dec = self.decoder(dec)
        return dec

# Initialize model, loss function, and optimizer
model = UNet3D(in_channels=1, out_channels=1).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_function = nn.BCELoss()

# Training loop
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for images, masks in train_loader:
        images, masks = images.to(DEVICE), masks.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, masks)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {epoch_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), os.path.join(BASE_DIR, "unet_heart_segmentation_3d.pth"))

# --- Inference on Patient Image ---
def predict_patient_image(model, patient_image_path, output_mask_path, device):
    model.eval()
    # Load and preprocess patient image
    img, _ = nrrd.read(patient_image_path)  # Read .nrrd file
    img = (img - img.min()) / (img.max() - img.min())  # Normalize the image
    img = scipy.ndimage.zoom(img, [float(dim) / img.shape[i] for i, dim in enumerate(INPUT_SHAPE)])
    img = torch.tensor(img, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)  # Add channel dimension

    with torch.no_grad():
        # Predict segmentation mask
        pred = model(img)
        pred = (pred > 0.5).float().cpu().numpy().squeeze()  # Apply threshold

    # Save the predicted mask as a NIfTI image
    pred_img = nib.Nifti1Image(pred, np.eye(4))
    nib.save(pred_img, output_mask_path)
    print(f"Prediction saved to: {output_mask_path}")

# Predict on the single patient image
predict_patient_image(model, PATIENT_IMAGE_PATH, OUTPUT_MASK_PATH, DEVICE)


Epoch 1/20, Loss: 13.6793
Epoch 2/20, Loss: 9.9353
Epoch 3/20, Loss: 6.0051
Epoch 4/20, Loss: 4.8690
Epoch 5/20, Loss: 4.7674
Epoch 6/20, Loss: 4.5850
Epoch 7/20, Loss: 4.5716
Epoch 8/20, Loss: 4.5396
Epoch 9/20, Loss: 4.7467
Epoch 10/20, Loss: 4.4343
Epoch 11/20, Loss: 4.4589
Epoch 12/20, Loss: 4.3445
Epoch 13/20, Loss: 4.3669
Epoch 14/20, Loss: 4.3203
Epoch 15/20, Loss: 4.3363
Epoch 16/20, Loss: 4.3019
Epoch 17/20, Loss: 4.2591
Epoch 18/20, Loss: 4.2115
Epoch 19/20, Loss: 4.2134
Epoch 20/20, Loss: 4.1573
Prediction saved to: C:\Users\ISU\cv_dataset\slice_data\segmented_mask.nii.gz
